In [4]:
import cma
es = cma.CMAEvolutionStrategy(
    [0.5, 1, 1, 0, 1, 0, 1],
    0.5,
    {'verb_log': 1, 'verb_disp': 1, 'seed': 42})

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=42, Tue Jul 29 18:41:08 2025)


In [1]:
import melign

dataset = melign.Dataset("dataset_test")

In [ ]:
import numpy as np

def fitness(code: tuple[float, float, float, float, float, float, float]) -> float:
    e, v, m, n, s, b, a = code
    weights: melign.align.score.MelodicsWeights = {
        "error": e,
        "velocity": v,
        "miss": m,
        "note_mean": n,
        "shadow": s,
        "between": b,
        "above_between": a,
    }
    score_model = melign.align.score.MelodicsModel(weights)
    results = melign.dev.search_min_score(
        dataset=dataset,
        candidates_dir="candidates",
        min_length=None,
        search_range=list(float(i) for i in np.arange(1, 14.1, 0.5)),
        score_models=[score_model],
        verbose=False,
        n_jobs=200
    )
    return 1 - results[0]["f1"]

def fitness_batch(codes: list[tuple[float, float, float, float, float, float, float]]) -> list[float]:
    models = []
    for e, v, m, n, s, b, a in codes:
        weights: melign.align.score.MelodicsWeights = {
            "error": e,
            "velocity": v,
            "miss": m,
            "note_mean": n,
            "shadow": s,
            "between": b,
            "above_between": a,
        }
        models.append(melign.align.score.MelodicsModel(weights))
    results = melign.dev.search_min_score(
        dataset=dataset,
        candidates_dir="candidates",
        min_length=None,
        search_range=list(float(i) for i in np.arange(1, 14.1, 0.5)),
        score_models=models,
        verbose=False,
        n_jobs=200
    )
    return [1 - r["f1"] for r in results]

In [ ]:
while not es.stop():
    solutions = es.ask()
    es.tell(solutions, fitness_batch(solutions))
    es.logger.add()  # write data to disc to be plotted
    es.disp()

Min score (n_jobs=200):   0%|          | 0/186 [00:00<?, ?it/s]

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 4.484168787127762e-02 1.0e+00 4.30e-01  4e-01  4e-01 7:38.7


Min score (n_jobs=200):   0%|          | 0/186 [00:00<?, ?it/s]